## PTSBE end-to-end workflow

**PTSBE** (Pre-Trajectory Sampling with Batch Execution) is a method for sampling from **noisy** quantum circuits efficiently. Instead of simulating the full density matrix and sampling once per shot, PTSBE:

1. **Traces** the kernel to get the gate sequence and qubit layout.
2. **Extracts noise sites** by matching the noise model to the trace (each noisy gate becomes a *noise site* with a set of Kraus outcomes, e.g. I, X, Y, Z for depolarization).
3. **Generates trajectories** — each trajectory is one possible *realization* of noise (one Kraus outcome per site). A **sampling strategy** decides which trajectories to use (e.g. **Exhaustive**: all combinations, or **Probabilistic**: sample by probability).
4. **Allocates shots** across trajectories (e.g. **proportional** to trajectory probability, or **uniform**).
5. **Runs batches** — for each trajectory, the circuit is run as a **noiseless** circuit with that trajectory's outcomes applied; results are collected.
6. **Aggregates** all per-trajectory counts into a single `SampleResult`.

You get the same statistics as density-matrix sampling (in the limit of many shots), but with the ability to batch many shots per trajectory and to control cost via the number of trajectories. This notebook runs the full workflow with a single API call: `cudaq.ptsbe.sample()`.

### 1. Set up the environment

Use the density-matrix simulator target (required for PTSBE). Set a random seed for reproducibility.

In [5]:
import cudaq

cudaq.set_target("density-matrix-cpu")
cudaq.set_random_seed(42)

### 2. Define the circuit and noise model

Define a kernel and attach a noise model. Each gate you add to the noise model becomes a **noise site** when that gate appears in the circuit. Here we use a small Bell-style circuit with depolarization on the Hadamard and on the controlled-X.

In [6]:
@cudaq.kernel
def bell_with_noise():
    q = cudaq.qvector(2)
    h(q[0])
    x.ctrl(q[0], q[1])
    mz(q)

noise = cudaq.NoiseModel()
noise.add_channel("h", [0], cudaq.DepolarizationChannel(0.05))
noise.add_channel("x", [0], cudaq.DepolarizationChannel(0.03))

### 3. Run PTSBE sampling

Call `cudaq.ptsbe.sample()` with the kernel, noise model, and shot count. Optional arguments:

- **sampling_strategy** — how trajectories are chosen:
  - **ExhaustiveSamplingStrategy()**: use all possible trajectories (every combination of Kraus outcomes per noise site).
  - **ProbabilisticSamplingStrategy(seed=...)**: sample trajectories randomly according to their probabilities; use a seed for reproducibility.
  - **OrderedSamplingStrategy()**: use the top-$k$ trajectories by probability (highest first), up to `max_trajectories`.
- **shot_allocation** — how shots are split across the chosen trajectories:
  - **PROPORTIONAL** (default): allocate shots in proportion to each trajectory’s probability.
  - **UNIFORM**: give each trajectory the same number of shots.
  - **LOW_WEIGHT_BIAS**: bias more shots toward low-weight (fewer errors) trajectories; optional `bias_strength` (default 2.0).
  - **HIGH_WEIGHT_BIAS**: bias more shots toward high-weight trajectories; optional `bias_strength` (default 2.0).
  Example: `ShotAllocationStrategy(type=cudaq.ptsbe.ShotAllocationType.UNIFORM)` or `ShotAllocationStrategy(type=cudaq.ptsbe.ShotAllocationType.LOW_WEIGHT_BIAS, bias_strength=6.0)`.
- **max_trajectories**: cap the number of trajectories (useful for large shot counts).

In [ ]:
shots = 1000000

strategy = cudaq.ptsbe.ProbabilisticSamplingStrategy(seed=42)
result = cudaq.ptsbe.sample(
    bell_with_noise,
    noise_model=noise,
    shots_count=shots,
    sampling_strategy=strategy,
)

print("PTSBE sample result:")
print(result)
print(f"Total shots: {result.get_total_shots()}")

PTSBE sample result:
{ 00:499857 11:500143 }

Total shots: 1000000


### 4. Compare with standard (density-matrix) sampling

To verify that PTSBE matches the usual noisy simulation, run standard `cudaq.sample()` with the same kernel and noise model. With enough shots, the two outcome distributions should be close (see the **PTSBE accuracy validation** example for a Hellinger fidelity comparison).

In [8]:
result_standard = cudaq.sample(bell_with_noise, noise_model=noise, shots_count=shots)
print("Standard (density-matrix) sample result:")
print(result_standard)
print(f"Total shots: {result_standard.get_total_shots()}")

Standard (density-matrix) sample result:
{ 00:499866 11:500134 }

Total shots: 1000000
